# Script content

1. **Load the 4 lists of recordings with each of the preprocessing steps (Raw, Filtered, ASR, and ICA)**:
   - The recordings are grouped by the preprocessing methods applied to the EEG data: Raw, Filtered, ASR, and ICA. Each list contains the corresponding processed data.

2. **Create the segmentation of the EEG recordings (1, 2, 3, 4, 5, 20)**:
   - We segment the EEG recordings into multiple time windows or chunks: 1 chunk, 2 chunks, 3 chunks, 4 chunks, 5 chunks, and 20 chunks, to allow feature extraction on smaller time slices.

3. **Extract the features for each matrix and save them**:
   - For each segmented matrix, we extract various features (e.g., statistical, frequency-domain features) and store them. We process all the 4 preprocessing types, across 35 different matrices (time segments) and for 19 channels.
   - This results in `4 x 35 x 19` matrices in total.

4. **Standardize the matrices**:
   - After extracting the features, we apply standardization to the feature matrices to ensure that all features have zero mean and unit variance.

5. **Test the normality, homoscedasticity, and conduct the population contrasts**:
   - We apply statistical tests to check for normality and homoscedasticity of the feature data.
   - We then split the results into three different lists:
     - `significant_diffs`: Contains the cases with statistically significant differences.
     - `indeterminate_cases`: Contains the cases where there was indecision in the statistical results.
     - `no_significant_diff_cases`: Contains the cases with no statistically significant differences.

6. **Analyze the different matrices**:
   - We analyze the feature matrices in terms of statistical significance, population contrasts, and other metrics, to extract meaningful patterns from the EEG data.


### Load the necessary libraries

In [1]:
from CollectData import CollectData
from CreateChunks import CreateChunks
import numpy as np
import pandas as pd
import mne
from CreateFeatures import CreateFeatures
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import pickle

### We retrieve the Raw, Filtered, ASR and ICA matrices

In [2]:
# Instantiate the CollectData object
collect_data = CollectData()

# Load all data (raw, filtered, ASR, ICA)
collect_data.load_data()

# Retrieve the lists with different preprocessing types
# List without preprocessing (raw data)
list_raw = collect_data.list_raw
# List with filtered data
list_filtered = collect_data.list_filtered
# List with ASR algorithm applied data
list_asr = collect_data.list_asr
# List with ICA removed non-brain sources data
list_ica = collect_data.list_ica 

# Save the names of the channels (you already have them manually listed)
chan_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
              'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz']


As the raw individuals do not match the Filtered, ASR and ICA, we reordered them.

In [3]:
# Create a dictionary that maps the index of the filtered data (key) to the corresponding raw data index (value)
dict_filt_raw = {}

# Iterate through the filtered and raw data lists to find matching shapes
for i in range(len(list_filtered)):  # Iterate over the filtered data list
    for j in range(len(list_raw)):   # Iterate over the raw data list
        if list_filtered[i].shape == list_raw[j].shape:  # Check if the shapes of the datasets match
            dict_filt_raw[i] = j  # Store the mapping of filtered index to raw index

# Reorder the raw data list based on the new indices (as per filtered data)
# Create a new list with the same length as the raw data list
reordered_raw_list = [None] * len(list_raw)

# Use the dictionary to reorder the raw data
for new_index, current_index in dict_filt_raw.items():
    reordered_raw_list[new_index] = list_raw[current_index]

## We show graphically the different preprocesses

In [ ]:
# Set MNE logging level to 'WARNING' to suppress unnecessary messages
mne.set_log_level('WARNING')

# Define the subject index
subj = 1

# Create a pandas DataFrame from the raw data for the subject
# Each column in the DataFrame corresponds to an EEG channel
data = pd.DataFrame(reordered_raw_list[subj], columns=chan_names)

# Define the sampling frequency (in Hz)
sfreq = 128

# Create an MNE Info object, which contains metadata about the EEG channels
# ch_names are the names of the channels (column names from the DataFrame)
# ch_types is set to 'eeg' since all channels are EEG
info = mne.create_info(ch_names=data.columns.to_list(), sfreq=sfreq, ch_types='eeg')

# Create a standard EEG montage (standard 10-20 system) for electrode placement
montage = mne.channels.make_standard_montage('standard_1020')

# Create a RawArray object in MNE using the EEG data and the info object
# Data is transposed because MNE expects channels as rows and time points as columns
raw = mne.io.RawArray(data.transpose(), info)

# Set the montage (standard electrode positions) for the Raw object
raw.set_montage(montage)

# Uncomment these lines if you want to inspect the raw data and info
# print(raw)
# print(raw.info)


We create a function which, starting from the subject, gives us the 4 images

In [5]:
def info_data(data_type):
    """
    Creates an MNE RawArray object from a pandas DataFrame containing EEG data.
    
    Args:
        data_type (pd.DataFrame): A DataFrame containing EEG data where columns are channels.
        
    Returns:
        raw (mne.io.RawArray): An MNE RawArray object containing the EEG data with the appropriate montage and info.
    """
    # Sampling frequency (in Hz)
    sfreq = 128
    
    # Create MNE Info object with channel names and sampling frequency
    info = mne.create_info(ch_names=data_type.columns.to_list(), sfreq=sfreq, ch_types='eeg')
    
    # Create a standard EEG montage (10-20 electrode system)
    montage = mne.channels.make_standard_montage('standard_1020')
    
    # Create an MNE RawArray using the transposed data (MNE expects channels as rows, time as columns)
    raw = mne.io.RawArray(data_type.transpose(), info)
    
    # Set the montage (electrode positions) for the RawArray object
    raw.set_montage(montage)
    
    return raw

def plots_preprocessing_subject(subj):
    """
    Loads the raw, filtered, ASR-filtered, and ICA-filtered data for a specific subject,
    creates MNE RawArray objects for each type of data, and returns them.
    
    Returns:
        tuple: A tuple containing RawArray objects for raw, filtered, ASR-filtered, and ICA-filtered data.
    """
    # Load raw data for the specific subject and create RawArray
    data_raw = pd.DataFrame(reordered_raw_list[subj], columns=chan_names)
    raw_raw = info_data(data_raw)
    
    # Load filtered data for the specific subject and create RawArray
    data_filt = pd.DataFrame(list_filtered[subj], columns=chan_names)
    raw_filt = info_data(data_filt)
    
    # Load ASR-filtered data for the specific subject and create RawArray
    data_asr = pd.DataFrame(list_asr[subj], columns=chan_names)
    raw_asr = info_data(data_asr)
    
    # Load ICA-filtered data for the specific subject and create RawArray
    data_ica = pd.DataFrame(list_ica[subj], columns=chan_names)
    raw_ica = info_data(data_ica)
    
    return raw_raw, raw_filt, raw_asr, raw_ica

Here is a plot of what the raw data looks like.

In [ ]:
data_raw = pd.DataFrame(list_ica[71],columns=chan_names)
raw_ica = info_data(data_raw)
fig = raw_ica.plot(color='#5AB0B0',scalings=dict(eeg=600), duration=100,\
             show_scrollbars=False, show_scalebars=False)

In [ ]:
# Generate and preprocess data for a specific subject (subject 71)
raw_raw, raw_filt, raw_asr, raw_ica = plots_preprocessing_subject(71)

# Frequency range for TFR computation (Morlet wavelet method)
freqs = np.arange(0.5, 60, 2)  # Frequency range from 0.5 to 60 Hz in 2 Hz steps

# Common parameters for plotting
plot_params = {
    'picks': ['Fp1'],  # Select the Fp1 channel for visualization
    'cmap': 'jet',  # Use the 'jet' color map
    'dB': True,  # Convert power to decibels
    'vmin': 0,  # Minimum value for color scaling
    'vmax': 80,  # Maximum value for color scaling
    'tmax': 80  # Maximum time to plot (in seconds)
}

# Compute and plot TFR for raw data (units = V^2/Hz)
power_raw = raw_raw.compute_tfr(method='morlet', freqs=freqs, picks=chan_names)
power_raw.plot(**plot_params)

# Compute and plot TFR for filtered data (units = V^2/Hz)
power_filt = raw_filt.compute_tfr(method='morlet', freqs=freqs, picks=chan_names)
power_filt.plot(**plot_params)

# Compute and plot TFR for ASR-filtered data (units = V^2/Hz)
power_asr = raw_asr.compute_tfr(method='morlet', freqs=freqs, picks=chan_names)
power_asr.plot(**plot_params)

# Compute and plot TFR for ICA-filtered data (units = V^2/Hz)
power_ica = raw_ica.compute_tfr(method='morlet', freqs=freqs, picks=chan_names)
power_ica.plot(**plot_params)


In [ ]:
# Retrieve the data for subject 71 with different preprocessing methods
raw_raw, raw_filt, raw_asr, raw_ica = plots_preprocessing_subject(71)

# List of raw data objects and corresponding titles for better identification in the plots
data_list = [
    (raw_raw, "Raw Data"),
    (raw_filt, "Filtered Data"),
    (raw_asr, "ASR Data"),
    (raw_ica, "ICA Data")
]

# Loop over the data and plot the PSD for each preprocessing step
for raw_data, title in data_list:
    raw_data.plot_psd(fmax=55, show=False)  # Plot up to 55 Hz
    plt.title(title)  # Add title to each plot for clarity
    plt.show()  # Display each plot individually


### We split the matrices into segments

In [9]:
# Instantiate the CreateChunks object to create chunks
create_chunks = CreateChunks()

# Create lists that will contain the chunks for each type of preprocessed data
# Each call to get_more_chunks splits the data into chunks with sizes [1, 2, 3, 4, 5, 20]
list_chunks_raw = create_chunks.get_more_chunks(list_raw)
list_chunks_filtered = create_chunks.get_more_chunks(list_filtered)
list_chunks_asr = create_chunks.get_more_chunks(list_asr)
list_chunks_ica = create_chunks.get_more_chunks(list_ica)

# Explanation of the structure:
# - Each of the above lists (lista_chunks_raw, lista_chunks_filtered, etc.) contains 6 elements.
# - These 6 elements correspond to the different chunk sizes: [1, 2, 3, 4, 5, 20].
# - Inside each of these elements, there are 121 subjects (or individuals).
# - For each subject, there's a 2D array of shape (time x channels), representing the EEG data.

### Extraction of the EEG features

In [10]:
def create_features_matrix(list_data):
    """
    This function extracts features from multiple EEG data chunks and saves each 
    chunk's features into a .npy file. The features are computed for each chunk 
    and stored in a final list.

    Args:
        list_data (list of lists): A list containing multiple chunks of EEG data 
                                   (each chunk is a list of individual windows).

    Returns:
        final_list (list): A list of numpy arrays, each containing the features extracted
                           for a chunk of EEG data.
    """
    # Instantiate the feature extraction object
    createFeatures = CreateFeatures()
    final_list = []  # List to store the feature matrices
    
    # Loop over the chunks (1-6 as per your chunk sizes)
    for chunk_idx, chunks in enumerate(list_data):
        print(f'Processing chunk group {chunk_idx + 1} of {len(list_data)}')

        # Loop over each chunk in the chunk group (for example, [1, 2, 3, 4, 5, 20])
        for chunk_inner_idx, chunk in enumerate(chunks):
            print(f'Processing inner chunk {chunk_inner_idx + 1} of {len(chunks)}')

            # Extract features for the current chunk
            array_data = createFeatures.get_features(chunk)

            # Get the current time to create a unique filename
            current_time = datetime.now().strftime("%d-%H-%M-%S")
            
            # Save the feature matrix as a .npy file
            np.save(f'array_features_{current_time}.npy', array_data)
            
            # Append the feature matrix to the final list
            final_list.append(array_data)
    
    return final_list


# Since this process is very resource-intensive, we decided to periodically save the feature arrays as they were being generated. In the next step, these arrays are read and combined into a single list, which we call `total_list`.

In [11]:
# # List of preprocessed data chunks for each preprocessing type
# data_chunks_list = [
#     ('raw', list_chunks_raw),
#     ('filtered', list_chunks_filtered),
#     ('asr', list_chunks_asr),
#     ('ica', list_chunks_ica)
# ]

# # Dictionary to store the resulting feature lists for each preprocessing type
# feature_lists = {}

# # Loop over the data chunks and process each
# for name, data_chunks in data_chunks_list:
#     print(f"Processing features for {name} data")
#     feature_lists[name] = create_features_matrix(data_chunks)

# # Now `feature_lists` contains the feature lists for 'raw', 'filtered', 'asr', and 'ica'
# # You can access them as follows:
# lists_feat_raw = feature_lists['raw']
# lists_feat_filtered = feature_lists['filtered']
# lists_feat_asr = feature_lists['asr']
# lists_feat_ica = feature_lists['ica']

For each data list (raw, filtered, ASR, and ICA), we have 35 matrices. Each matrix contains the features of 121 individuals, extracted from different segments of the recording:

1. 100% of the recording (1/1)
2. First half of the recording (1st/2)
3. Second half of the recording (2nd/2)
4. First third of the recording (1st/3)
5. Second third of the recording (2nd/3)
6. Third third of the recording (3rd/3)
...
33. Eighteenth twentieth of the recording (18th/20)
34. Nineteenth twentieth of the recording (19th/20)
35. Twentieth twentieth of the recording (20th/20)

### We read the saved arrays
#### It is not necessary

In [12]:
# lists_feat_raw=['array_features/array_features_06-10-11-28.npy', 'array_features/array_features_06-10-23-37.npy',
#                'array_features/array_features_06-10-35-34.npy', 'array_features/array_features_06-10-41-49.npy',
#                'array_features/array_features_06-10-47-55.npy', 'array_features/array_features_06-10-53-57.npy',
#                'array_features/array_features_06-10-57-54.npy', 'array_features/array_features_06-11-01-50.npy', 
#                'array_features/array_features_06-11-05-42.npy', 'array_features/array_features_06-11-09-29.npy', 
#                'array_features/array_features_06-11-12-12.npy', 'array_features/array_features_06-11-14-55.npy', 
#                'array_features/array_features_06-11-17-40.npy', 'array_features/array_features_06-11-20-24.npy', 
#                'array_features/array_features_06-11-23-07.npy', 'array_features/array_features_06-11-23-32.npy', 
#                'array_features/array_features_06-11-23-56.npy', 'array_features/array_features_06-11-24-19.npy', 
#                'array_features/array_features_06-11-24-42.npy', 'array_features/array_features_06-11-25-05.npy', 
#                'array_features/array_features_06-11-25-28.npy', 'array_features/array_features_06-11-25-51.npy', 
#                'array_features/array_features_06-11-26-15.npy', 'array_features/array_features_06-11-26-38.npy', 
#                'array_features/array_features_06-11-27-01.npy', 'array_features/array_features_06-11-27-24.npy', 
#                'array_features/array_features_06-11-27-47.npy', 'array_features/array_features_06-11-28-10.npy', 
#                'array_features/array_features_06-11-28-33.npy', 'array_features/array_features_06-11-28-56.npy', 
#                'array_features/array_features_06-11-29-19.npy', 'array_features/array_features_06-11-29-42.npy', 
#                'array_features/array_features_06-11-30-04.npy', 'array_features/array_features_06-11-30-27.npy', 
#                'array_features/array_features_06-11-30-50.npy']

# lists_feat_fil=['array_features/array_features_06-12-16-45.npy', 'array_features/array_features_06-12-30-31.npy',
#                'array_features/array_features_06-12-44-03.npy', 'array_features/array_features_06-12-51-01.npy',
#                'array_features/array_features_06-12-57-51.npy', 'array_features/array_features_06-13-04-41.npy',
#                'array_features/array_features_06-13-09-04.npy', 'array_features/array_features_06-13-13-24.npy', 
#                'array_features/array_features_06-13-17-42.npy', 'array_features/array_features_06-13-21-58.npy', 
#                'array_features/array_features_06-13-25-01.npy', 'array_features/array_features_06-13-28-01.npy', 
#                'array_features/array_features_06-13-31-01.npy', 'array_features/array_features_06-13-34-00.npy', 
#                'array_features/array_features_06-13-36-59.npy', 'array_features/array_features_06-13-37-23.npy', 
#                'array_features/array_features_06-13-37-48.npy', 'array_features/array_features_06-13-38-12.npy', 
#                'array_features/array_features_06-13-38-37.npy', 'array_features/array_features_06-13-39-01.npy', 
#                'array_features/array_features_06-13-39-25.npy', 'array_features/array_features_06-13-39-50.npy', 
#                'array_features/array_features_06-13-40-14.npy', 'array_features/array_features_06-13-40-38.npy', 
#                'array_features/array_features_06-13-41-03.npy', 'array_features/array_features_06-13-41-27.npy', 
#                'array_features/array_features_06-13-41-51.npy', 'array_features/array_features_06-13-42-15.npy', 
#                'array_features/array_features_06-13-42-40.npy', 'array_features/array_features_06-13-43-04.npy', 
#                'array_features/array_features_06-13-43-28.npy', 'array_features/array_features_06-13-43-53.npy', 
#                'array_features/array_features_06-13-44-17.npy', 'array_features/array_features_06-13-44-41.npy', 
#                'array_features/array_features_06-13-45-05.npy']

# lists_feat_asr=['array_features/array_features_06-14-27-03.npy', 'array_features/array_features_06-14-40-07.npy',
#                'array_features/array_features_06-14-53-09.npy', 'array_features/array_features_06-14-59-43.npy',
#                'array_features/array_features_06-15-06-50.npy', 'array_features/array_features_06-15-15-31.npy',
#                'array_features/array_features_06-15-20-58.npy', 'array_features/array_features_06-15-26-25.npy', 
#                'array_features/array_features_06-15-31-50.npy', 'array_features/array_features_06-15-36-47.npy', 
#                'array_features/array_features_06-15-39-42.npy', 'array_features/array_features_06-15-42-37.npy', 
#                'array_features/array_features_06-15-45-31.npy', 'array_features/array_features_06-15-48-25.npy', 
#                'array_features/array_features_06-15-51-18.npy', 'array_features/array_features_06-15-51-43.npy', 
#                'array_features/array_features_06-15-52-08.npy', 'array_features/array_features_06-15-52-32.npy', 
#                'array_features/array_features_06-15-52-57.npy', 'array_features/array_features_06-15-53-21.npy', 
#                'array_features/array_features_06-15-53-46.npy', 'array_features/array_features_06-15-54-10.npy', 
#                'array_features/array_features_06-15-54-35.npy', 'array_features/array_features_06-15-54-59.npy', 
#                'array_features/array_features_06-15-55-23.npy', 'array_features/array_features_06-15-55-48.npy', 
#                'array_features/array_features_06-15-56-12.npy', 'array_features/array_features_06-15-56-36.npy', 
#                'array_features/array_features_06-15-57-01.npy', 'array_features/array_features_06-15-57-25.npy', 
#                'array_features/array_features_06-15-57-50.npy', 'array_features/array_features_06-15-58-14.npy', 
#                'array_features/array_features_06-15-58-39.npy', 'array_features/array_features_06-15-59-03.npy',
#                'array_features/array_features_06-15-59-27.npy']  

# lists_feat_ica=['array_features/array_features_06-16-44-17.npy', 'array_features/array_features_06-17-00-47.npy',
#                'array_features/array_features_06-17-17-17.npy', 'array_features/array_features_06-17-25-40.npy',
#                'array_features/array_features_06-17-33-38.npy', 'array_features/array_features_06-17-40-05.npy',
#                'array_features/array_features_06-17-44-07.npy', 'array_features/array_features_06-17-48-09.npy', 
#                'array_features/array_features_06-17-52-10.npy', 'array_features/array_features_06-17-56-12.npy', 
#                'array_features/array_features_06-17-59-01.npy', 'array_features/array_features_06-18-01-50.npy', 
#                'array_features/array_features_06-18-04-39.npy', 'array_features/array_features_06-18-07-29.npy', 
#                'array_features/array_features_06-18-10-18.npy', 'array_features/array_features_06-18-10-42.npy', 
#                'array_features/array_features_06-18-11-06.npy', 'array_features/array_features_06-18-11-30.npy', 
#                'array_features/array_features_06-18-11-54.npy', 'array_features/array_features_06-18-12-18.npy', 
#                'array_features/array_features_06-18-12-42.npy', 'array_features/array_features_06-18-13-06.npy', 
#                'array_features/array_features_06-18-13-30.npy', 'array_features/array_features_06-18-13-54.npy', 
#                'array_features/array_features_06-18-14-18.npy', 'array_features/array_features_06-18-14-43.npy', 
#                'array_features/array_features_06-18-15-07.npy', 'array_features/array_features_06-18-15-31.npy', 
#                'array_features/array_features_06-18-15-55.npy', 'array_features/array_features_06-18-16-19.npy', 
#                'array_features/array_features_06-18-16-43.npy', 'array_features/array_features_06-18-17-07.npy', 
#                'array_features/array_features_06-18-17-31.npy', 'array_features/array_features_06-18-17-55.npy', 
#                'array_features/array_features_06-18-18-19.npy',]         

In [13]:
# def load_npy(file_list):
#     """
#     Loads multiple .npy files from a list of filenames and returns them as a list of arrays.

#     Args:
#         file_list (list of str): List of filenames (paths) to the .npy files to be loaded.

#     Returns:
#         list of numpy arrays: A list containing the numpy arrays loaded from the provided filenames.
#     """
#     loaded_arrays = []
    
#     # Loop over each filename in the provided list
#     for filename in file_list:
#         try:
#             # Load the .npy file and append the resulting array to the list
#             array = np.load(filename)
#             loaded_arrays.append(array)
#         except Exception as e:
#             print(f"Error loading {filename}: {e}")
    
#     return loaded_arrays

In [14]:
# # Initialize an empty list to store all loaded arrays
# total_list = []

# # Loop through each list of feature files (raw, filtered, ASR, and ICA)
# for feature_list in [lists_feat_raw, lists_feat_fil, lists_feat_asr, lists_feat_ica]:
#     # Load the numpy arrays from each list of filenames and append them to total_list
#     loaded_features = load_npy(feature_list)
#     total_list.append(loaded_features)

In [15]:
# # Saving the total_list to a file using pickle
# with open('total_list.pkl', 'wb') as f:
#     pickle.dump(total_list, f)

### We load the saved list

In [16]:
# Loading the total_list from the saved file
with open('total_list.pkl', 'rb') as f:
    total_list = pickle.load(f)

### standardisation of matrices

We need to standardise the data by columns for the test to be correct.

In [18]:
# Initialize an empty list to hold the standardized data for all preprocessing types
total_list_stand = []  # 4 lists (raw, filtered, ASR, ICA)

# Loop through each preprocessed list in total_list (raw, filtered, ASR, ICA)
for list_prep in total_list:
    lista_trozos = []  # This will store the 35 matrices for each preprocessed type

    # Loop through the 35 matrices inside each preprocessed list
    for mat in list_prep:
        lista_canales = []  # List to store standardized data for 19 channels
        
        # Standardize each channel (121x53 for each channel)
        for chan in range(19):
            # Convert to DataFrame for easier manipulation and standardization
            df = pd.DataFrame(mat[chan])
            
            # Standardize the data using StandardScaler
            scaler_standard = StandardScaler()
            arr_stand = scaler_standard.fit_transform(df)

            # Convert the standardized data back to a DataFrame
            df_stand = pd.DataFrame(arr_stand)

            # Add the label column (1 for subjects < 61, and 0 for subjects >= 61)
            labels = [1 if x < 61 else 0 for x in range(121)]
            df_stand['Label'] = labels  # Add the labels as a new column

            # Rename the columns for clarity (53 feature columns + 1 label column)
            column_names = [f'Feature_{x}' for x in range(53)]
            column_names.append('Label')
            df_stand.columns = column_names

            # Append the standardized DataFrame for this channel to lista_canales
            lista_canales.append(df_stand)

        # Append the list of standardized channels for this matrix (35 matrices) to lista_trozos
        lista_trozos.append(lista_canales)

    # Append the processed data for each preprocessing type (raw, filtered, ASR, ICA) to total_list_stand
    total_list_stand.append(lista_trozos)


In [49]:
# Saving the total_list_stand to a file using pickle
with open('total_list_stand.pkl', 'wb') as f:
    pickle.dump(total_list_stand, f)

You have 4 main lists in total_list (raw, filtered, ASR, ICA). For each, you process 35 matrices, each representing different divisions of the recordings (e.g., 1/1, 1/2, etc.).
Within each matrix, there are 19 channels, and you standardize the data for each channel (121x53 matrix) using StandardScaler.
After standardization, you add a label for each subject (121 rows), indicating whether they belong to group 1 or 0, and then you append the standardized data to total_list_stand.

Now total_list_stand contains standardized features with labels for each preprocessing type:

- total_list_stand[0] -> Standardized data for raw preprocessing
- total_list_stand[1] -> Standardized data for filtered preprocessing
- total_list_stand[2] -> Standardized data for ASR preprocessing
- total_list_stand[3] -> Standardized data for ICA preprocessing

Now, total_list_stand contains 4 lists, one for each preprocessing type. Each of these 4 lists contains 35 sublists, representing the different segments we generated. Within each of these 35 sublists, there are 19 matrices (one per channel), each with dimensions 121x54.

### Normality test, test of variances and population contrast.

In [ ]:
from scipy import stats

# Counters for different cases (sum total = 2660)
count_norm_homo = 0  # Normal distribution, homogeneous variances
count_norm_hete = 0  # Normal distribution, heterogeneous variances
count_non_norm_homo = 0  # Non-normal distribution, homogeneous variances
count_non_norm_hete = 0  # Non-normal distribution, heterogeneous variances
count_sig_diff = 0  # Significant differences between groups
count_no_sig_diff = 0  # No significant differences between groups

# Lists to store tuples with significant differences (i, j, k, s)
# (preprocessing, segment, channel, feature)
significant_diffs = []
indeterminate_cases = []
no_significant_diff_cases = []

# Iterate over the preprocessing types
for preprocessing_idx in range(4):
    # Iterate over the recording segments
    for segment_idx in range(35):
        # Iterate over the channels
        for channel_idx in range(19):
            # Iterate over the features
            df_features = total_list_stand[preprocessing_idx][segment_idx][channel_idx]
            for feature_idx in range(53):
                # Separate populations (ADHD vs Control)
                adhd_group = df_features[df_features['Label'] == 1]
                control_group = df_features[df_features['Label'] == 0]
                
                # Normality test
                pval_control = stats.normaltest(control_group.iloc[:, feature_idx]).pvalue
                pval_adhd = stats.normaltest(adhd_group.iloc[:, feature_idx]).pvalue

                if (pval_adhd < 0.05) or (pval_control < 0.05):
                    # Non-normal samples: Levene's test for variance
                    pval_var = stats.levene(adhd_group.iloc[:, feature_idx], control_group.iloc[:, feature_idx]).pvalue

                    if pval_var < 0.05:
                        # Heterogeneous variances: indeterminate case
                        count_non_norm_hete += 1
                        indeterminate_cases.append((preprocessing_idx, segment_idx, channel_idx, feature_idx))
                    else:
                        # Homogeneous variances: Perform T-test or Mann-Whitney test
                        pval_ttest = stats.ttest_ind(adhd_group.iloc[:, feature_idx], control_group.iloc[:, feature_idx]).pvalue
                        count_non_norm_homo += 1
                        if pval_ttest > 0.05:
                            # No significant differences -> Means are equal
                            count_no_sig_diff += 1
                            no_significant_diff_cases.append((preprocessing_idx, segment_idx, channel_idx, feature_idx))
                        else:
                            # Significant differences exist
                            count_sig_diff += 1
                            significant_diffs.append((preprocessing_idx, segment_idx, channel_idx, feature_idx))
                else:
                    # Normal samples: Bartlett's test for variance
                    pval_var = stats.bartlett(adhd_group.iloc[:, feature_idx], control_group.iloc[:, feature_idx]).pvalue

                    if pval_var < 0.05:
                        # Heterogeneous variances
                        count_norm_hete += 1
                        pval_ttest = stats.ttest_ind(adhd_group.iloc[:, feature_idx], control_group.iloc[:, feature_idx], equal_var=False).pvalue
                        if pval_ttest > 0.05:
                            # No significant differences -> Means are equal
                            count_no_sig_diff += 1
                            no_significant_diff_cases.append((preprocessing_idx, segment_idx, channel_idx, feature_idx))
                        else:
                            # Significant differences exist
                            count_sig_diff += 1
                            significant_diffs.append((preprocessing_idx, segment_idx, channel_idx, feature_idx))
                    else:
                        # Homogeneous variances: T-test
                        count_norm_homo += 1
                        pval_ttest = stats.ttest_ind(adhd_group.iloc[:, feature_idx], control_group.iloc[:, feature_idx]).pvalue
                        if pval_ttest > 0.05:
                            # No significant differences -> Means are equal
                            count_no_sig_diff += 1
                            no_significant_diff_cases.append((preprocessing_idx, segment_idx, channel_idx, feature_idx))
                        else:
                            # Significant differences exist
                            count_sig_diff += 1
                            significant_diffs.append((preprocessing_idx, segment_idx, channel_idx, feature_idx))

# Total cases calculated
total_cases = count_norm_homo + count_norm_hete + count_non_norm_homo + count_non_norm_hete
indeterminate_cases_count = total_cases - count_no_sig_diff - count_sig_diff

# Output results
print('Total case count: ', total_cases)
print('Significant difference case count:', count_sig_diff)
print('No significant difference case count:', count_no_sig_diff, len(no_significant_diff_cases))
print('Cases with non-normal distribution and heterogeneous variance:', indeterminate_cases_count, len(indeterminate_cases))
print('Length of significant difference cases list:', len(significant_diffs))
print('Indeterminate significant cases:', count_sig_diff - len(significant_diffs))

### Individual analysis of results

In [ ]:
# Create a DataFrame with the statistically significant cases
df_significant = pd.DataFrame(significant_diffs)

# Define column names to clearly represent what each column contains
columns = ['Preprocessing', 'Segment', 'Channel', 'Feature']
df_significant.columns = columns

# Plot the distribution of significant cases across preprocessing types
df_significant['preprocessing'].value_counts().plot(kind='bar')

# Display the bar plot
plt.xlabel('Preprocessing Type')
plt.ylabel('Count of Significant Cases')
plt.show()

In [ ]:
# Create a DataFrame with the indeterminate cases
df_indeterminate_cases = pd.DataFrame(indeterminate_cases)

# Define column names for the DataFrame: preprocessing step, segment, channel, and feature
column_names = ['Preprocessing', 'Segment', 'Channel', 'Feature']
df_indeterminate_cases.columns = column_names

# Plot the counts of indeterminate cases per preprocessing type
df_indeterminate_cases['Preprocessing'].value_counts().plot(kind='bar', title='Indeterminate Cases by Preprocessing Type')

# Display the bar plot
plt.xlabel('Preprocessing Type')
plt.ylabel('Count of Indeterminate Cases')
plt.show()


In [ ]:
# Create a DataFrame with the no significant difference cases
df_no_significant_diff = pd.DataFrame(no_significant_diff_cases)

# Define column names for the DataFrame: preprocessing step, segment, channel, and feature
column_names = ['Preprocessing', 'Segment', 'Channel', 'Feature']
df_no_significant_diff.columns = column_names

# Plot the counts of no significant difference cases per preprocessing type
df_no_significant_diff['Preprocessing'].value_counts().plot(kind='bar', title='No Significant Difference Cases by Preprocessing Type')

# Display the bar plot
plt.xlabel('Preprocessing Type')
plt.ylabel('Count of No Significant Difference Cases')
plt.show()

In [ ]:
# Get the count of statistically significant cases per segment
segment_counts = df_significant['Segment'].value_counts().sort_index()
segments = segment_counts.index
x_pos = np.arange(len(segments))

# Define custom colors for each segment
colors = ['black', 'red', 'red', 'green', 'green', 'green',
          'blue', 'blue', 'blue', 'blue', 'cyan', 'cyan',
          'cyan', 'cyan', 'cyan', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow']

# Create the bar plot
plt.bar(x_pos, segment_counts, color=colors)

# Set the x-ticks and their labels
plt.xticks(x_pos, segments)

# Add labels and title to the plot
plt.xlabel('Recording Segment (Trozo)')
plt.ylabel('Count of Statistically Significant Cases')
plt.title('Statistically Significant Cases by Segment')

# Display the plot
plt.show()

In [ ]:
# Get the count of indeterminate cases per segment
segment_counts = df_indeterminate_cases['Segment'].value_counts().sort_index()
segments = segment_counts.index
x_pos = np.arange(len(segments))

# Define custom colors for each segment
colors = ['black', 'red', 'red', 'green', 'green', 'green',
          'blue', 'blue', 'blue', 'blue', 'cyan', 'cyan',
          'cyan', 'cyan', 'cyan', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow']

# Create the bar plot
plt.bar(x_pos, segment_counts, color=colors)

# Set the x-ticks and their labels
plt.xticks(x_pos, segments)

# Add labels and title to the plot
plt.xlabel('Recording Segment')
plt.ylabel('Count of Indeterminate Cases')
plt.title('Indeterminate Cases by Segment')

# Display the plot
plt.show()

In [ ]:
# Get the count of "no significant difference" cases per segment
segment_counts = df_no_significant_diff['Segment'].value_counts().sort_index()
segments = segment_counts.index
x_pos = np.arange(len(segments))

# Define custom colors for each segment
colors = ['black', 'red', 'red', 'green', 'green', 'green',
          'blue', 'blue', 'blue', 'blue', 'cyan', 'cyan',
          'cyan', 'cyan', 'cyan', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow', 'yellow', 'yellow', 
          'yellow', 'yellow', 'yellow']

# Create the bar plot
plt.bar(x_pos, segment_counts, color=colors)

# Set the x-ticks and their labels
plt.xticks(x_pos, segments)

# Add labels and title to the plot
plt.xlabel('Recording Segment (Trozo)')
plt.ylabel('Count of No Significant Difference Cases')
plt.title('No Significant Difference Cases by Segment')

# Display the plot
plt.show()

In [37]:
# Define the mapping of channel indices to channel names
channel_mapping = {
    0: 'Fp1', 1: 'Fp2', 2: 'F3', 3: 'F4', 4: 'C3', 5: 'C4',
    6: 'P3', 7: 'P4', 8: 'O1', 9: 'O2', 10: 'F7', 11: 'F8',
    12: 'T7', 13: 'T8', 14: 'P7', 15: 'P8', 16: 'Fz', 17: 'Cz', 18: 'Pz'
}

In [ ]:
# Count the occurrences of significant cases per channel
channel_counts = df_significant['Channel'].value_counts().sort_index()

# Replace channel indices with their corresponding names
channel_counts.index = channel_counts.index.map(channel_mapping)

# Plot the bar chart with channel names on the x-axis
channel_counts.plot(kind='bar', title='Significant Cases by Channel')

# Add labels and title
plt.xlabel('Channel')
plt.ylabel('Count of Significant Cases')
plt.show()


In [ ]:
# Count the occurrences of indeterminate cases per channel
channel_counts = df_indeterminate_cases['Channel'].value_counts().sort_index()

# Replace channel indices with their corresponding names
channel_counts.index = channel_counts.index.map(channel_mapping)

# Plot the bar chart with channel names on the x-axis
channel_counts.plot(kind='bar', title='Indeterminate Cases by Channel')

# Add labels and title
plt.xlabel('Channel')
plt.ylabel('Count of Indeterminate Cases')
plt.show()

In [ ]:
# Count the occurrences of "no significant difference" cases per channel
channel_counts = df_no_significant_diff['Channel'].value_counts().sort_index()

# Replace channel indices with their corresponding names
channel_counts.index = channel_counts.index.map(channel_mapping)

# Plot the bar chart with channel names on the x-axis
channel_counts.plot(kind='bar', title='No Significant Difference Cases by Channel')

# Add labels and title
plt.xlabel('Channel')
plt.ylabel('Count of No Significant Difference Cases')
plt.show()

In [43]:
# Define the mapping of column indices to feature names
feature_mapping = {
    0: 'mean', 1: 'variance', 2: 'std', 3: 'ptp_amp', 4: 'skewness', 5: 'kurtosis', 
    6: 'rms', 7: 'quantile', 8: 'hurst_exp', 9: 'app_entropy', 10: 'decorr_time', 
    11: 'pow_freq_bands_1', 12: 'pow_freq_bands_2', 13: 'pow_freq_bands_3', 14: 'pow_freq_bands_4', 
    15: 'hjorth_mobility_spect', 16: 'hjorth_complexity_spect', 17: 'hjorth_mobility', 
    18: 'hjorth_complexity', 19: 'higuchi_fd', 20: 'katz_fd', 21: 'zero_crossings', 
    22: 'line_length', 23: 'spect_slope_1', 24: 'spect_slope_2', 25: 'spect_slope_3', 
    26: 'spect_slope_4', 27: 'spect_entropy', 28: 'energy_freq_bands_1', 
    29: 'energy_freq_bands_2', 30: 'energy_freq_bands_3', 31: 'energy_freq_bands_4', 
    32: 'spect_edge_freq', 33: 'wavelet_coef_energy_1', 34: 'wavelet_coef_energy_2', 
    35: 'wavelet_coef_energy_3', 36: 'wavelet_coef_energy_4', 37: 'wavelet_coef_energy_5', 
    38: 'wavelet_coef_energy_6', 39: 'teager_kaiser_energy_1', 40: 'teager_kaiser_energy_2', 
    41: 'teager_kaiser_energy_3', 42: 'teager_kaiser_energy_4', 43: 'teager_kaiser_energy_5', 
    44: 'teager_kaiser_energy_6', 45: 'teager_kaiser_energy_7', 46: 'teager_kaiser_energy_8', 
    47: 'teager_kaiser_energy_9', 48: 'teager_kaiser_energy_10', 49: 'teager_kaiser_energy_11', 
    50: 'teager_kaiser_energy_12', 51: 'teager_kaiser_energy_13', 52: 'teager_kaiser_energy_14'
}

In [ ]:
# Count the occurrences of significant cases per feature column
feature_counts = df_significant['Feature'].value_counts().sort_index()

# Replace the column indices with their corresponding feature names
feature_counts.index = feature_counts.index.map(feature_mapping)

# Plot the bar chart with feature names on the x-axis
feature_counts.plot(kind='bar', title='Significant Cases by Feature')

# Add labels and title
plt.xlabel('Feature')
plt.ylabel('Count of Significant Cases')
plt.show()

In [ ]:
# Count the occurrences of indeterminate cases per feature column
feature_counts = df_indeterminate_cases['Feature'].value_counts().sort_index()

# Replace the column indices with their corresponding feature names
feature_counts.index = feature_counts.index.map(feature_mapping)

# Plot the bar chart with feature names on the x-axis
feature_counts.plot(kind='bar', title='Indeterminate Cases by Feature')

# Add labels and title
plt.xlabel('Feature')
plt.ylabel('Count of Indeterminate Cases')
plt.show()

In [ ]:
# Count the occurrences of "no significant difference" cases per feature column
feature_counts = df_no_significant_diff['Feature'].value_counts().sort_index()

# Replace the column indices with their corresponding feature names
feature_counts.index = feature_counts.index.map(feature_mapping)

# Plot the bar chart with feature names on the x-axis
feature_counts.plot(kind='bar', title='No Significant Difference Cases by Feature')

# Add labels and title
plt.xlabel('Feature')
plt.ylabel('Count of No Significant Difference Cases')
plt.show()